`requirements/gpu.txt`

In [27]:
import numpy as np, jsonpickle
from qiskit_aer.noise import NoiseModel
from qiskit_aer import AerSimulator
from qiskit_experiments.library import QuantumVolume

In [45]:
def load_noisy_simulator_from_calibration(path):
    with open(path, "r") as f:
        payload = jsonpickle.decode(f.read())

    noise_model = NoiseModel.from_dict(payload["noise_model"])
    coupling_map = payload.get("coupling_map")

    coupling_map = payload.get("coupling_map")

    return AerSimulator(
        noise_model=noise_model,
        coupling_map=coupling_map,
        basis_gates=noise_model.basis_gates,
        method="automatic",
        device="GPU",
    )

In [46]:
def run_qv_for_subset(backend, subset, shots=2000, trials=100, seed=42):
    ideal_backend = AerSimulator(method="statevector", device="GPU")

    exp = QuantumVolume(
        physical_qubits=subset,
        trials=trials,
        seed=seed,
        simulation_backend=ideal_backend,
    )

    exp.set_run_options(shots=shots)

    exp.set_transpile_options(
        layout_method="trivial",
        routing_method="basic",
        optimization_level=0,
        seed_transpiler=seed,
        num_processes=1,
    )

    return exp.run(backend).block_for_results()

In [47]:
def run_qv_for_computer(calibration_path, subset, shots=2000, trials=100, seed=42):
    backend = load_noisy_simulator_from_calibration(calibration_path)
    return run_qv_for_subset(backend, subset, shots=shots, trials=trials, seed=seed)

In [51]:
exp_data = run_qv_for_computer(
    "calibrations/ibm_marrakesh/20260129_101824.json",
    subset=[0, 1, 2, 3, 4, 5, 6],
    shots=2000,
    trials=100,
)

/tmp/ipykernel_37586/2859233853.py:5: DeprecationWarning: from_dict has been deprecated as of qiskit-aer 0.15.0 and will be removed no earlier than 3 months from that release date.
  noise_model = NoiseModel.from_dict(payload["noise_model"])


In [52]:
df = exp_data.analysis_results(dataframe=True)
df


,name,experiment,components,value,quality,backend,run_time,HOPs,two_sigma,depth,trials,success,confidence
4b7f17a5,mean_HOP,QuantumVolume,"[Q0, Q1, Q2, Q3, Q4, Q5, Q6]",0.78+/-0.04,good,aer_simulator,None,"[0.78, 0.794, 0.792, 0.778, 0.8005, 0.7745, 0....",0.082256,7,100,None,None
8e89d550,quantum_volume,QuantumVolume,"[Q0, Q1, Q2, Q3, Q4, Q5, Q6]",128,good,aer_simulator,None,None,None,7,100,True,0.997889
